In [5]:
# 📦 Required installs
# !pip install langchain openai faiss-cpu

from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS, Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

import os
from dotenv import load_dotenv

# 2️⃣ Load API keys
load_dotenv(".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 1. Load the text file
with open("sample1.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 2. Split the text into chunks
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_text(raw_text)
documents = [Document(page_content=chunk) for chunk in chunks]

# 3. Create vector store
embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embedding=embedding)

# 4. Setup LLM
llm = ChatOpenAI(temperature=0)


In [6]:
#Get retrieves --> it does not generate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# Compressor using LLM
compressor = LLMChainExtractor.from_llm(llm)

# Create compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectorstore.as_retriever()
)

# Run retrieval
print("🔹 Contextual Compression Results:")
results = compression_retriever.get_relevant_documents("Who created LangChain?")
for doc in results:
    print("-", doc.page_content)


🔹 Contextual Compression Results:
- LangChain was created by Harrison Chase.


In [7]:
#Integration with ConversationalRetrievalChain- Without agent mode
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# ✅ ContextualCompressionRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectorstore.as_retriever()
)

# ✅ Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ✅ Chain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=compression_retriever,
    memory=memory
)

# 🧪 Ask a few questions
print("🔹 ConversationalRetrievalChain:")
print(rag_chain.invoke({"question": "What is LangChain?"})["answer"])
print(rag_chain.invoke({"question": "Who created it?"})["answer"])


C:\Users\mojayara\AppData\Local\Temp\ipykernel_23364\2540500264.py:16: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


🔹 ConversationalRetrievalChain:
LangChain is a framework for building applications with Large Language Models (LLMs).
LangChain was created by Harrison Chase.


In [8]:
#Integration into Agent (with Tool)

from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

# 5. Wrap RAG as a StructuredTool
def rag_tool_fn(question: str) -> str:
    return rag_chain.invoke({
        "question": question,
        "chat_history": [memory.chat_memory.messages ]
    })["answer"]

# Structured Tool
rag_tool = StructuredTool.from_function(
    name="RAG_Tool",
    description="Answer LangChain-related questions with context.",
    func=rag_tool_fn
)

# Agent with memory
agent = initialize_agent(
    tools=[rag_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# 🧪 Ask via agent
print("\n🔹 Agent Conversation:")
print(agent.run("What is LangChain?"))
print(agent.run("Who created it?"))


C:\Users\mojayara\AppData\Local\Temp\ipykernel_23364\4200800445.py:21: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
C:\Users\mojayara\AppData\Local\Temp\ipykernel_23364\4200800445.py:31: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(agent.run("What is LangChain?"))



🔹 Agent Conversation:


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: RAG_Tool
Action Input: What is LangChain?
Observation: LangChain was created by Harrison Chase.
Thought:Do I need to use a tool? No
AI: LangChain was created by Harrison Chase.

> Finished chain.
LangChain was created by Harrison Chase.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: LangChain was created by Harrison Chase.

> Finished chain.
LangChain was created by Harrison Chase.


In [9]:
#multiQuery
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import RetrievalQA

# MultiQueryRetriever
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm
)

# RetrievalQA Chain
rag_multi = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=multi_query_retriever,
    return_source_documents=True
)

# 🧪 Ask question
#print("\n🔹 RAG Pipeline (MultiQueryRetriever):")
#res = rag_multi.run("Tell me about LangChain creator and features.")
#print(res)

# 🧪 Ask question
print("\n🔹 RAG Pipeline (MultiQueryRetriever):")
res = rag_multi.invoke({"query": "Tell me about LangChain creator and features."})

print("Answer:", res["result"])
print("\nSources:")
for doc in res["source_documents"]:
    print("-", doc.page_content[:200])  # print first 200 chars of each doc




🔹 RAG Pipeline (MultiQueryRetriever):
Answer: LangChain was created by Harrison Chase. It is a framework for building applications with LLMs. LangChain supports RAG, agents, memory, tools, and more. It is commonly used in chatbots, document Q&A, and AI workflow.

Sources:
- LangChain is a framework for building applications with LLMs.LangChain was created by Harrison Chase.LangChain supports RAG, agents, memory, tools, and more.It’s commonly used in chatbots, document Q&
- In the quiet village of Thendralpur, an old clockmaker named Kavin worked alone in a tiny workshop filled with ticking clocks. One evening, a young girl found a broken pocket watch with a strange glow
